In [4]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import librosa

# Load the Whisper model and processor from Hugging Face
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def transcribe_audio(file_path):
    # Load and process the audio file
    audio_data, sampling_rate = librosa.load(file_path, sr=16000)
    audio_input = processor(audio_data, sampling_rate=sampling_rate, return_tensors="pt").input_features

    # Generate transcription
    with torch.no_grad():
        generated_ids = model.generate(audio_input)
        transcript = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return transcript

# Test the speech-to-text function on an audio file
if __name__ == '__main__':
    audio_file_path = "test2.wav"
    
    # Call the transcription function
    transcript = transcribe_audio(audio_file_path)
    
    print("Transcript:", transcript)

Transcript:  This is test 2 for audio to speech
